# 第３章　音源分離で用いる数学的知識の基礎

## 第１節　音源分離で用いる線形代数

### 行列の基本的な演算

In [16]:
import numpy as np 

A=np.matrix([[3.,1.,2.],[2.,3.,1.]])
B=np.matrix([[4.,2.],[-1.,3.],[1.,5.]])
C=np.matrix([[-1.,2.,4.],[1.,8.,6]])
D=np.matrix([[3.,1.+2j,2.+3.j],[2.,3.-4.j,1.+3j]])
E=np.matrix([[4.+4.j,2.-3.j],[-1.+1.j,3.-2.j],[1.+3.j,5.+5.j]])

# 行列Aに行列Bを掛ける
print("AB=\n",np.matmul(A,B))
print("AB=\n",np.einsum("mk,kn->mn",A,B))   # アインシュタインの縮約記法

# 行列AとCのアダマール積
print("A*B=\n",np.multiply(A,C))

# 行列Aの転置
print("A^T=\n",A.T)
print("A^T=\n",np.transpose(A,axes=(1,0)))
print("A^T=\n",np.swapaxes(A,1,0))

# 複素行列のエルミート転置
print("D^H=\n",D.H) # X.H: Xの複素共役転置（エルミート転置）を返す
print("D^H=\n",np.swapaxes(np.conjugate(D),1,0))    # np.conjugate(X): Xの複素共役を返す

# 行列の積に対するエルミート転置
print("(DE)^H=\n",np.matmul(D,E).H)
print("D^H E^H=\n",np.matmul(E.H,D.H))

# 単位行列の定義
I=np.eye(N=3)
print("I=\n",I)


AB=
 [[13. 19.]
 [ 6. 18.]]
AB=
 [[13. 19.]
 [ 6. 18.]]
A*B=
 [[-3.  2.  8.]
 [ 2. 24.  6.]]
A^T=
 [[3. 2.]
 [1. 3.]
 [2. 1.]]
A^T=
 [[3. 2.]
 [1. 3.]
 [2. 1.]]
A^T=
 [[3. 2.]
 [1. 3.]
 [2. 1.]]
D^H=
 [[3.-0.j 2.-0.j]
 [1.-2.j 3.+4.j]
 [2.-3.j 1.-3.j]]
D^H=
 [[3.-0.j 2.-0.j]
 [1.-2.j 3.+4.j]
 [2.-3.j 1.-3.j]]
(DE)^H=
 [[ 2.-20.j  1.-21.j]
 [ 8.-20.j -5. +4.j]]
D^H E^H=
 [[ 2.-20.j  1.-21.j]
 [ 8.-20.j -5. +4.j]]
I=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


### アインシュタインの縮約記法

※numpyではeinsum()で定義されている.  
$$
C_{lk}=A_{lk}B_k
$$
行列$A_{lk}$を$M$行$R$列、$B_k$を$R$行$N$列の行列とする.このとき要素ごとに書くと$C$の$m$行$n$列の要素は、
$$
C_{lkmn}=\sum_{r}A_{lkmr}B_{krn}
$$
行列の積を取る計算は要素の５つのインデックス$l,k,m,r,n$の中の一つについて和を取る計算と見ることができる.  
アインシュタインの縮約記法では、どのように掛け算を行うかの計算のルールを入力のインデックスと出力のインデックスの形式を与えることで教える.  
上記式の例では、"$lkmr,krn\rightarrow lkmn$"と書く.  
  
行列$C$を計算する際に、行列$A_{lk}$と$B_k$をかけたものを時間方向に和を取るように計算したい場合がある.
$$
C_{k}=\sum_{l}A_{lk}B_{k}
$$
この場合は、
$$
C_{kmn}=\sum_{l,k}A_{lkmr}B_{krn}
$$
となるため、アインシュタインの縮約記法では$lkmr,krn\rightarrow kmn$と書けば良い.  
  
アダマール積もアインシュタインの縮約記法で記述可能.
$$
C_{lkmn}=A_{lkmn}B_{kmn}
$$
この掛け算は$lkmn,kmn\rightarrow lkmn$と書ける.
  
アインシュタイン縮約記法の掛け算は任意の数の入力テンソルを受け付けることができる.
$$
D_{k}=\sum_{l}A_{lk}B_{k}C_{l}
$$
という計算を行う場合は"$lkmr,krs,lsn->kmn$"と書ける.

### アインシュタインの縮約記法を使ったテンソル演算

In [12]:
import numpy as np 

np.random.seed(0)

# テンソルの大きさを設定
L=10
K=5
M=3
R=3
N=3

# ランダムな複素数のテンソルを生成
A=np.random.uniform(size=(L*K*M*R))+np.random.uniform(size=(L*K*M*R))*1.j
A=np.reshape(A,(L,K,M,R))

B=np.random.uniform(size=(K*R*N))+np.random.uniform(size=(K*R*N))*1.j
B=np.reshape(B,(K,R,N))

# einsumを使って行列積を計算する
C=np.einsum("lkmr,krn->lkmn",A,B)

# 行列Cの大きさを表示
print("shape(C): ",np.shape(C))

# l=0,k=0の要素について検算実施
print("A(0,0)B(0,0)=\n",np.matmul(A[0,0,...],B[0,...]))
print("C(0,0)=\n",C[0,0,...])

# einsumを使って行列積をl,kについて計算したあと、かつl方向に和を取る
C=np.einsum("lkmr,krn->kmn",A,B)

# 行列Cの大きさを表示
print("shape(C): ",np.shape(C))

# k=0の要素について検算実施
for l in range(L):
    if l==0:
        C_2=np.matmul(A[l,0,...],B[0,...])
    else:
        C_2=C_2+np.matmul(A[l,0,...],B[0,...])

print("C_2(0)=\n",C_2)
print("C(0)=\n",C[0,...])

# einsumを使ってアダマール積を計算する
C=np.einsum("lkmn,kmn->lkmn",A,B)

# 行列Cの大きさを表示
print("shape(C): ",np.shape(C))

# l=0,k=0の要素について検算実施
print("A(0,0)×B(0,0)=\n",np.multiply(A[0,0,...],B[0,...]))
print("C(0,0)=\n",C[0,0,...])

shape(C):  (10, 5, 3, 3)
A(0,0)B(0,0)=
 [[-0.49715528+1.60739814j -0.0430694 +2.13098867j  0.49058998+1.53302537j]
 [-0.40774965+1.27706091j -0.11232905+1.78980525j  0.30717926+1.39995875j]
 [-0.44493154+2.37398823j  0.59400714+1.73358933j  0.62369562+0.94900167j]]
C(0,0)=
 [[-0.49715528+1.60739814j -0.0430694 +2.13098867j  0.49058998+1.53302537j]
 [-0.40774965+1.27706091j -0.11232905+1.78980525j  0.30717926+1.39995875j]
 [-0.44493154+2.37398823j  0.59400714+1.73358933j  0.62369562+0.94900167j]]
shape(C):  (5, 3, 3)
C_2(0)=
 [[-3.961214  +14.08164414j -0.02424697+16.96557508j
   3.57699604+11.04347156j]
 [-5.11705802+15.03435616j -0.89518857+18.20029068j
   2.40292593+12.18576631j]
 [-5.42651683+15.55774111j -0.25094293+15.58870789j
   2.05990148+10.16543559j]]
C(0)=
 [[-3.961214  +14.08164414j -0.02424697+16.96557508j
   3.57699604+11.04347156j]
 [-5.11705802+15.03435616j -0.89518857+18.20029068j
   2.40292593+12.18576631j]
 [-5.42651683+15.55774111j -0.25094293+15.58870789j
   2.0599

## 第２節　逆行列

正方行列$A$に対して以下の関係を持つ行列$B$を逆行列と呼び、このような逆行列を$A^{-1}$と書く.
$$
AB=BA=I
$$
逆行列は行列ごとに２つ以上存在しない.逆行列が転置行列と一致する場合、そのような行列を直交行列と呼ぶ.
直交行列では、
$$
AA^T=I
$$
が成立する.同様に、逆行列がエルミート転置行列と一致する場合、そのような行列をユニタリ行列と呼ぶ.ユニタリ行列では、
$$
AA^H=I
$$
が成立する.  
２行２列の正方行列Aの行列式は
$$
\det A =a_{1,1}a_{2,2}-a_{1,2}a_{2,1}
$$
で計算できる.
数学的にM行M列の正方行列Aの行列
式は
$$
\det A = \sum_{\sigma \in S_M} \mathrm{sgn} (\sigma)\prod_{i=1}^M a_{i,\sigma(i)}
$$
と表すことができる.  
  
行列$A$をスカラー$c$倍した行列$cA$に対する行列式は以下の様に計算できる.
$$
\det cA=c^M \det A
$$
その他便利な公式
$$
\det AB=\det A \det B\\
\det A^T = \det A\\
\det A^H = (\det A)^*
$$

### 逆行列演算

In [11]:
import numpy as np 

np.random.seed(0)

L=10
M=3
N=3

A=np.random.uniform(size=(L*M*N))+np.random.uniform(size=(L*M*N))*1.j
A=np.reshape(A,(L,M,N))

# 行列式を計算する
detA=np.linalg.det(A)
print("detA(0):\n",detA[0])

# すべての要素を３倍した行列の行列式を計算する
det3A=np.linalg.det(3*A)
print("det3A(0):\n",det3A[0])

# 逆行列を計算する
A_inv=np.linalg.inv(A)  # 行列式が0の場合はエラーとなる

# Aとかけて単位行列になっているか確認
AA_inv=np.einsum("lmk,lkn->lmn",A,A_inv)
print("単位行列？:\n",AA_inv[0])

A_invA=np.einsum("lmk,lkn->lmn",A_inv,A)
print("単位行列？:\n",A_invA[0])

# 一般化逆行列計算
A_inv=np.linalg.pinv(A) # 行列式が0の場合でも0に近い擬似逆行列を算出して出力する

# Aとかけて単位行列になっているか確認
AA_inv=np.einsum("lmk,lkn->lmn",A,A_inv)
print("単位行列？:\n",AA_inv[0])

A_invA=np.einsum("lmk,lkn->lmn",A_inv,A)
print("単位行列？:\n",A_invA[0])

detA(0):
 (0.5026136129571367+0.020264008179737707j)
det3A(0):
 (13.570567549842691+0.5471282208529202j)
単位行列？:
 [[ 1.00000000e+00+0.00000000e+00j  2.22044605e-16-1.11022302e-16j
  -5.55111512e-17+1.11022302e-16j]
 [ 1.11022302e-16+0.00000000e+00j  1.00000000e+00+1.66533454e-16j
   0.00000000e+00+0.00000000e+00j]
 [ 2.22044605e-16+0.00000000e+00j  2.22044605e-16+4.44089210e-16j
   1.00000000e+00-1.11022302e-16j]]
単位行列？:
 [[ 1.00000000e+00+0.00000000e+00j -8.32667268e-17+0.00000000e+00j
   1.66533454e-16-3.33066907e-16j]
 [ 1.11022302e-16-8.32667268e-17j  1.00000000e+00+0.00000000e+00j
   0.00000000e+00+5.55111512e-17j]
 [ 1.11022302e-16+1.11022302e-16j  1.11022302e-16+2.22044605e-16j
   1.00000000e+00+1.11022302e-16j]]
単位行列？:
 [[1.00000000e+00+2.77555756e-16j 8.88178420e-16-1.16573418e-15j
  2.22044605e-16+6.66133815e-16j]
 [2.22044605e-16-3.33066907e-16j 1.00000000e+00+2.77555756e-16j
  2.22044605e-16+2.22044605e-16j]
 [7.21644966e-16-2.22044605e-16j 2.22044605e-16+5.55111512e-16j
  1

### 行列とベクトルの掛け算

In [13]:
import numpy as np 

A=np.matrix([[3.,1.,2.],[2.,3.,1.]])
b=np.array([2.,1.,4.])

print("A=\n",A)
print("b=\n",b)

# 行列Aにベクトルbを掛ける
print("Ab=\n",np.dot(A,b))

A=
 [[3. 1. 2.]
 [2. 3. 1.]]
b=
 [2. 1. 4.]
Ab=
 [[15. 11.]]


$M$個の要素をもつベクトル$b$とベクトル$c$の内積を次のように定義する.ベクトル$b$が実ベクトルの場合は
$$
d=b^Tc=\sum_{m=1}^{M}b_m c_m
$$
$b$と$c$が複素数のベクトルの場合は、$b^T$の代わりにエルミート転置$b^H$を用いて
$$
d=b^Hc=\sum_{m=1}^{M}b_m^* c_m
$$
ベクトル$b$と$c$の内積$d$が0になる場合、ベクトル$b$と$c$は「直交する」という.  
おなじベクトル$b$同士の内積計算は、実ベクトルの場合、
$$
d=b^Tb=\sum_{m=1}^{M}|b_m|^2
$$
このとき$\sqrt{d}$はベクトル$b$の長さ、大きさを表す量となる.  
信号処理では各要素を二乗して足したあと、$\sqrt{}$をとるとることで計算されるベクトルの大きさを$L_2$ノルムと呼ぶ.  
これから派生して$L_p$ノルムを以下のように定義する.
$$
\|b\|_p=\sqrt[p]{\sum_{m=1}^{M}|b_m|^p}
$$
$L_2$ノルムが0となるベクトルを0ベクトルと呼ぶ.また、$L_2$ノルムが1となるベクトルを単位ベクトルと呼ぶ.

### ベクトルの内積計算

In [33]:
import numpy as np 

a=np.matrix([3+2j,1-1j,2+2j])
b=np.matrix([2+5j,1-1j,4+1j])

#　ベクトルの内積計算
print("a^Hb=\n",np.inner(np.conjugate(a),b))
print("a^Ha=\n",np.inner(np.conjugate(a),a))
print("a^Ha=\n",np.dot(a,a.H))
print("a^Ha=\n",np.matmul(a,a.H))
print("a^Ha=\n",np.einsum("lm,mn->ln",a,a.H))

a^Hb=
 [[28.+5.j]]
a^Ha=
 [[23.+0.j]]
a^Ha=
 [[23.+0.j]]
a^Ha=
 [[23.+0.j]]
a^Ha=
 [[23.+0.j]]


正方行列$A$のトレース(tr:対角成分の和)を以下のように定義する.
$$
\mathrm{tr}(A)=\sum_{m=1}^M a_{m,m}
$$
行列の積$ABC$のトレースには以下の性質がある.
$$
\mathrm{tr}(ABC)=\mathrm{tr}(BCA)=\mathrm{tr}(CAB)
$$
この性質を利用すると、以下の変形が可能になる.ここで、$b$は$M$個の要素を持つベクトル、$A$は$M$行$M$列の正方行列とする.
$$
b^HAb=\mathrm{tr}(b^HAb)=\mathrm{tr}(Abb^H)
$$
その他トレース計算で有用な公式
$$
\mathrm{tr}(cA)=c\mathrm{tr}(A)\\
\mathrm{tr}(A+B)=\mathrm{tr}(A)+\mathrm{tr}(B)
$$

### 行列のトレース計算

In [39]:
import numpy as np 

np.random.seed(0)

L=10
M=3
N=3

A=np.random.uniform(size=(L*M*N))+np.random.uniform(size=(L*M*N))*1.j
A=np.reshape(A,(L,M,N))

b=np.random.uniform(size=(L*M))+np.random.uniform(size=(L*M))*1.j
b=np.reshape(b,(L,M))

# 行列Aのtrace
print("tr(A)=\n",np.trace(A,axis1=-2,axis2=-1))

# einsumを用いたトレース計算
print("tr(A)=\n",np.einsum("lmm->l",A))

# b^H,A,bの計算
print("b^H A b=\n",np.einsum("lm,lmn,ln->l",np.conjugate(b),A,b))
print("tr(A b b^H)=\n",np.trace(np.einsum("lmn,ln,lk->lmk",A,b,np.conjugate(b)),axis1=-2,axis2=-1))



tr(A)=
 [1.93613106+1.43691507j 2.141658  +1.53913593j 1.3829894 +1.80366411j
 2.3365381 +1.68899047j 1.63837034+1.20717354j 1.13639345+1.91421455j
 1.33151712+1.68292822j 0.70393543+1.63820299j 1.57755123+1.25480629j
 1.60894868+0.31010161j]
tr(A)=
 [1.93613106+1.43691507j 2.141658  +1.53913593j 1.3829894 +1.80366411j
 2.3365381 +1.68899047j 1.63837034+1.20717354j 1.13639345+1.91421455j
 1.33151712+1.68292822j 0.70393543+1.63820299j 1.57755123+1.25480629j
 1.60894868+0.31010161j]
b^H A b=
 [3.15053328+1.92248684j 2.45635919+2.48501598j 2.99216437+3.26370601j
 3.15903598+4.23799258j 2.38987666+2.08639805j 1.99005115+2.63773589j
 1.17483773+2.97727122j 0.89998733+0.96364299j 0.57263214+0.7107664j
 2.62964589+1.79840628j]
tr(A b b^H)=
 [3.15053328+1.92248684j 2.45635919+2.48501598j 2.99216437+3.26370601j
 3.15903598+4.23799258j 2.38987666+2.08639805j 1.99005115+2.63773589j
 1.17483773+2.97727122j 0.89998733+0.96364299j 0.57263214+0.7107664j
 2.62964589+1.79840628j]


### 固有値計算

In [50]:
import numpy as np 

np.random.seed(0)

L=10
M=3
N=3

A=np.random.uniform(size=(L*M*N))+np.random.uniform(size=(L*M*N))*1.j
A=np.reshape(A,(L,M,N))

# 正定エルミート行列を作る
B=np.einsum("lmk,lnk->lmn",A,np.conjugate(A))

# Aの固有値分解を実施
w,v=np.linalg.eig(A)

# 固有値と固有ベクトルからAを復元できるか検証
A_reconst=np.einsum("lmk,lk,lkn->lmn",v,w,np.linalg.inv(v))
print("A[0]:\n",A[0])
print("A_reconst[0]:\n",A_reconst[0])

# Bの固有値分解を実施
w,v=np.linalg.eigh(B)

# 固有値と固有ベクトルからBを復元できるか検証
B_reconst=np.einsum("lmk,lk,lnk->lmn",v,w,np.conjugate(v))
print("B[0]:\n",B[0])
print("B_reconst[0]:\n",B_reconst[0])

A[0]:
 [[0.5488135 +0.31856895j 0.71518937+0.66741038j 0.60276338+0.13179786j]
 [0.54488318+0.7163272j  0.4236548 +0.28940609j 0.64589411+0.18319136j]
 [0.43758721+0.58651293j 0.891773  +0.02010755j 0.96366276+0.82894003j]]
A_reconst[0]:
 [[0.5488135 +0.31856895j 0.71518937+0.66741038j 0.60276338+0.13179786j]
 [0.54488318+0.7163272j  0.4236548 +0.28940609j 0.64589411+0.18319136j]
 [0.43758721+0.58651293j 0.891773  +0.02010755j 0.96366276+0.82894003j]]
B[0]:
 [[1.74030925+0.j         1.43685044-0.16906931j 1.76831828+0.02566734j]
 [1.43685044+0.16906931j 1.5239999 +0.j         1.816471  -0.11543232j]
 [1.76831828-0.02566734j 1.816471  +0.11543232j 2.94693088+0.j        ]]
B_reconst[0]:
 [[1.74030925+0.00000000e+00j 1.43685044-1.69069308e-01j
  1.76831828+2.56673361e-02j]
 [1.43685044+1.69069308e-01j 1.5239999 -1.04083409e-17j
  1.816471  -1.15432321e-01j]
 [1.76831828-2.56673361e-02j 1.816471  +1.15432321e-01j
  2.94693088+0.00000000e+00j]]
